In [2]:
import os
import numpy as np
import pandas as pd
import math
import xml.etree.ElementTree as ET
import geopandas as gpd
import math
import matplotlib.pyplot as plt
import matplotlib as mpl
from collections import Counter
from geopandas import GeoDataFrame 

In [3]:
path = "G:/My Drive/2021/Bias/sumo_simulation/appsim"
os.chdir(path)
#os.listdir()

In [ ]:
#other method 
#considering taz as a point and construct a network
#possible resource
#https://github.com/sharifulgeo/ESRI/tree/master/Connect_Points

In [192]:
#os.listdir()

In [4]:
#get taz data
taz_tree = ET.parse('Taz_bigger_Seattle_all_fordtagg.add.xml')
taz_root = taz_tree.getroot()

#for i in root.iter('timestep'):
#    print(i.attrib)
##output:{'time': '0.00'} {'time': '300.00'}
#get net data
net_tree = ET.parse('test_2_traffic_sig_revise_5.net.xml')
net_root = net_tree.getroot()

In [5]:
for i in os.listdir():
    if 'fcd' in i and 'xml' in i:
        print(i)

fcd_output.xml
fcd_output_5.xml
fcd_output_5_30sec.xml
fcd_output_10per_all_30sec.xml
fcd_output_10per_ht_30sec.xml
fcd_output_10per_all_1sec.xml
fcd_output_10per_ht_1sec.xml
fcd_output_10per_all_30sec_dtSea.xml
fcd_output_10per_ht_30sec_dtSea.xml


In [9]:
#use the fcd output to check the data
#fcd_file = 'fcd_output_10per_all_30sec.xml'
#fcd_file = 'fcd_output_10per_ht_30sec.xml'
#fcd_file = 'fcd_output_10per_all_30sec_dtSea.xml'
fcd_file = 'fcd_output_10per_ht_30sec_dtSea.xml'

tree = ET.parse(fcd_file)
root = tree.getroot()
#for i in root.iter('timestep'):
#    print(i.attrib)
##output:{'time': '0.00'} {'time': '300.00'}

### Missing TAZs processing

In [10]:
#defined packages
def assign_val(missing_edge_connection, target_edge, connect_edge):
    if target_edge not in missing_edge_connection:
        missing_edge_connection[target_edge] = []
        missing_edge_connection[target_edge].append(connect_edge)
    else:
        if connect_edge not in missing_edge_connection[target_edge]:
            missing_edge_connection[target_edge].append(connect_edge)
    return(missing_edge_connection)

def convert2taz(connect_edges):
    connect_taz = []
    for edge in connect_edges:
        if edge in edge_taz_dict:
            connect_taz.append(edge_taz_dict[edge])
    include_taz = np.unique(connect_taz)
    taz_ct = []
    for taz in include_taz:
        taz_ct.append(connect_taz.count(taz))
    try:
        return(include_taz[taz_ct.index(max(taz_ct))])
    except:
        return(-1)
            
def sep_str(lane_str):
    if len(lane_str) != 0:
        return(lane_str.split(' '))
    else:
        return('1')
    
def get_edge(int_inc_lane, edge_net_lst):
    edge_cur = []
    for l in int_inc_lane:
        if l != '1':
            if l[-2] == '_':
                if l[:-2] in edge_net_lst:
                    edge_cur.append(l[:-2])
            if l[-3] == '_':
                if l[:-3] in edge_net_lst:
                    if l[:-3] not in edge_cur:
                        edge_cur.append(l[:-3])
    return(edge_cur)

def get_taz(edge, edge_taz_dict):
    try:
        return(edge_taz_dict[edge])
    except:
        return(0)
    
def assign_edge_val(edge_cur, junctionid, edge_taz_dict):
    edge_val = []
    for edge in edge_cur:
        val = get_taz(edge, edge_taz_dict)
        if val == 0:
            pass
        else:
            edge_val.append(val)
    edge_val.append(junctionid)
    return(edge_val)   
  
    
#get edge id in tazs
edge_taz_dict = {}
edges_in2tazs = {}
for tazs in taz_root.iter('taz'):
    taz_id_cur = tazs.attrib['id']
    for edge in tazs.findall('tazSource'):
        edge_id_cur = edge.attrib['id']
        if edge_id_cur not in edge_taz_dict:
            edge_taz_dict[edge_id_cur] = taz_id_cur
        else:
            #print('warning, edges appear in more than one taz')
            edges_in2tazs[edge_id_cur] = []
            edges_in2tazs[edge_id_cur].append(edge_taz_dict[edge_id_cur])
            edges_in2tazs[edge_id_cur].append(taz_id_cur)
            
print('edges in taz', edges_in2tazs)

#get fcd data
#check edges in the data
edge_lst = []
lane_lst = []

for t in root.findall('timestep'):
    #get lane id from vehicle
    for veh in t.findall('vehicle'):
        #print(veh.attrib['id'])
        lane_id_cur = veh.attrib['lane']
        #if '_' in lane_id_cur:
        #    lane_id_cur = ExtractEdgeFromLane(lane_id_cur)
        if lane_id_cur not in lane_lst:
            lane_lst.append(lane_id_cur)
    for per in t.findall('person'):
        edge_id_cur = per.attrib['edge']
        if edge_id_cur not in edge_lst:
            edge_lst.append(edge_id_cur)
            
#convert lane to edge from net file
lane_dict = {}
edge_net_lst = [] #get all edges from the network
for edge in net_root.findall('edge'):
    edge_id = edge.attrib['id']
    if edge_id not in edge_net_lst:
        edge_net_lst.append(edge_id)   
    for lane in edge.findall('lane'):
        lane_id = lane.attrib['id']
       # print(lane_id)
        if lane_id in lane_lst and lane_id not in lane_dict:
            lane_dict[lane_id] = edge_id
#print(lane_dict)

for edge in list(lane_dict.values()):
    if edge not in edge_lst:
        edge_lst.append(edge)
        

edges_missing_tazs = []
edges_with_tazs = list(edge_taz_dict.keys())
for edges in edge_lst:
    if edges not in edges_with_tazs and edges not in edges_missing_tazs:
        edges_missing_tazs.append(edges)
        
missings_taz_veh = []
missings_taz_ped_pub = []
for edges in edges_missing_tazs:
    if ":" in edges: 
        missings_taz_ped_pub.append(edges)
    else:
        missings_taz_veh.append(edges)
        
print('edge with tazs identified:', len(edges_with_tazs))
print('edge w.o tazs identified:', len(edges_missing_tazs))

#assign taz to missing edges
missing_edge_connection = {}
            
for c in net_root.findall('connection'):
    edge1 = c.attrib['from']
    edge2 = c.attrib['to']
    #print(edge1, edge2)
    
    if edge1 in edges_missing_tazs:
        #print(edge1)
        missing_edge_connection = assign_val(missing_edge_connection,
                                             edge1, edge2)
    elif edge2 in edges_missing_tazs:
        #print(edge2)
        missing_edge_connection = assign_val(missing_edge_connection,
                                            edge2, edge1)
        
#print(len(list(missing_edge_connection.keys())))
#print(len(edges_missing_tazs))
#previous return = 6329 (both)

print(len(list(missing_edge_connection.keys())))
print(len(edges_missing_tazs))
#previous return = 3203 (for 30 sec data)

#for edges in missing_edge_connection:
#print('test', convert2taz(missing_edge_connection['165086027#0']))

missing_edge_taz = {}
for edge in missing_edge_connection:
    missing_edge_taz[edge] = convert2taz(missing_edge_connection[edge])
    
#get edge with no taz mapped
edges_ = []
index_ = 0
for i in list(missing_edge_taz.values()):
    if i == -1:
        edges_.append(list(missing_edge_taz.keys())[index_])
    index_ += 1
    
for edge in missing_edge_taz:
    taz_val = missing_edge_taz[edge]
    if taz_val != -1:
        if edge not in edge_taz_dict:
            edge_taz_dict[edge] = taz_val
        else:
            print('edge in taz already', edge)   
            
#check '_' in lanes and edges
for i in edge_net_lst:
    if i[-2] == '_':
        pass
    if i[-3] == '_':
        pass
    if i[-4] == '_':
        print(i)


#get junction
missing_edge_junc = {}
#index_ = 0
for junc in net_root.findall('junction'):
    #print(index_)
    int_lane = junc.attrib['intLanes']
    inc_lane = junc.attrib['incLanes']
    junctionid = junc.attrib['id']

    int_inc_lane = []
    int_inc_lane.extend(sep_str(int_lane))
    int_inc_lane.extend(sep_str(inc_lane))
    #print(int_inc_lane)
    if len(int_inc_lane) != 0:
        edge_cur = get_edge(int_inc_lane, edge_net_lst)
    for edge in edge_cur:
        if edge in edges_:
            if edge not in missing_edge_junc:
                missing_edge_junc[edge] = assign_edge_val(edge_cur, 
                                                          junctionid, 
                                                          edge_taz_dict)            

print(len(missing_edge_junc))
selected_junc = list(missing_edge_junc.values())

index_ = 0
for edge in missing_edge_junc:
    tazs = missing_edge_junc[edge][:-1]
    #print(tazs)
    taz_ct = []
    unique_taz = np.unique(tazs)
    #print(unique_taz)
    for taz in unique_taz:
        taz_ct.append(tazs.count(taz))
    #print(taz_ct)
    try:
        taz_ = unique_taz[taz_ct.index(max(taz_ct))]
    except:
        taz_ = -1
    #print(taz_)
    if edge not in edge_taz_dict and taz_!=-1:
        edge_taz_dict[edge] = taz_
    elif taz_ == -1:
        print(edge)
#through the check, we assign 629 to the missing edges
#edge_taz_dict[':53217375_5'] = '629'

#missing_edge_junc[':53147426_0']
#edge_taz_dict[':cluster_gneJ449_gneJ451_1']
#edge_taz_dict[lane_dict['460423550#0_1']]

edges in taz {}
edge with tazs identified: 7220
edge w.o tazs identified: 9218
9218
9218
1704


### fcd data processing (ground truth with taz assigned)

In [11]:
#get total taz in sumo network
taz_lst = []
for taz in taz_root.findall('taz'):
    taz_lst.append(taz.attrib['id'])
    
#def get_dt(fcd_outputpath, data_path, lane_dict, edge_taz_dict):
    #use the fcd output to check the data
#using 30 sec as an example
#fcd_outputpath = 'fcd_output_10per_all_30sec.xml'
#fcd_outputpath = 'fcd_output_10per_ht_30sec.xml'
#fcd_file


write_line = ''

#tree = ET.parse(fcd_outputpath)
#root = tree.getroot()
with open (fcd_file[:-3]+'csv', 'w') as f:
    for t in root.findall('timestep'):
        time_cur = t.attrib['time']
        #print(time_cur)
        #get lane id from vehicle
        for veh in t.findall('vehicle'):
            vehid = veh.attrib['id']
            #print(vehid)
            lane_id_cur = veh.attrib['lane']
            taz_id = edge_taz_dict[lane_dict[lane_id_cur]]
            write_line = vehid+','+taz_id+','+time_cur+'\n'
            f.write(write_line)
        try:    
            for per in t.findall('person'):
                perid = per.attrib['id']
                #print(perid)
                edge_id_cur = per.attrib['edge']
                taz_id = edge_taz_dict[edge_id_cur]
                write_line = perid+','+taz_id+','+time_cur+'\n'
                f.write(write_line)
        except: 
            pass
f.close()
print('data output done')     

data output done


In [ ]:
#use the fcd output to check the data
fcd_file = 'fcd_output_10per_all_1sec.xml'
#fcd_file = 'fcd_output_10per_ht_1sec.xml'
tree = ET.parse(fcd_file)
root = tree.getroot()
#for i in root.iter('timestep'):
#    print(i.attrib)
##output:{'time': '0.00'} {'time': '300.00'}

#defined packages
def assign_val(missing_edge_connection, target_edge, connect_edge):
    if target_edge not in missing_edge_connection:
        missing_edge_connection[target_edge] = []
        missing_edge_connection[target_edge].append(connect_edge)
    else:
        if connect_edge not in missing_edge_connection[target_edge]:
            missing_edge_connection[target_edge].append(connect_edge)
    return(missing_edge_connection)

def convert2taz(connect_edges):
    connect_taz = []
    for edge in connect_edges:
        if edge in edge_taz_dict:
            connect_taz.append(edge_taz_dict[edge])
    include_taz = np.unique(connect_taz)
    taz_ct = []
    for taz in include_taz:
        taz_ct.append(connect_taz.count(taz))
    try:
        return(include_taz[taz_ct.index(max(taz_ct))])
    except:
        return(-1)
            
def sep_str(lane_str):
    if len(lane_str) != 0:
        return(lane_str.split(' '))
    else:
        return('1')
    
def get_edge(int_inc_lane, edge_net_lst):
    edge_cur = []
    for l in int_inc_lane:
        if l != '1':
            if l[-2] == '_':
                if l[:-2] in edge_net_lst:
                    edge_cur.append(l[:-2])
            if l[-3] == '_':
                if l[:-3] in edge_net_lst:
                    if l[:-3] not in edge_cur:
                        edge_cur.append(l[:-3])
    return(edge_cur)

def get_taz(edge, edge_taz_dict):
    try:
        return(edge_taz_dict[edge])
    except:
        return(0)
    
def assign_edge_val(edge_cur, junctionid, edge_taz_dict):
    edge_val = []
    for edge in edge_cur:
        val = get_taz(edge, edge_taz_dict)
        if val == 0:
            pass
        else:
            edge_val.append(val)
    edge_val.append(junctionid)
    return(edge_val)   
  
    
#get edge id in tazs
edge_taz_dict = {}
edges_in2tazs = {}
for tazs in taz_root.iter('taz'):
    taz_id_cur = tazs.attrib['id']
    for edge in tazs.findall('tazSource'):
        edge_id_cur = edge.attrib['id']
        if edge_id_cur not in edge_taz_dict:
            edge_taz_dict[edge_id_cur] = taz_id_cur
        else:
            #print('warning, edges appear in more than one taz')
            edges_in2tazs[edge_id_cur] = []
            edges_in2tazs[edge_id_cur].append(edge_taz_dict[edge_id_cur])
            edges_in2tazs[edge_id_cur].append(taz_id_cur)
            
print('edges in taz', edges_in2tazs)

#get fcd data
#check edges in the data
edge_lst = []
lane_lst = []

for t in root.findall('timestep'):
    #get lane id from vehicle
    for veh in t.findall('vehicle'):
        #print(veh.attrib['id'])
        lane_id_cur = veh.attrib['lane']
        #if '_' in lane_id_cur:
        #    lane_id_cur = ExtractEdgeFromLane(lane_id_cur)
        if lane_id_cur not in lane_lst:
            lane_lst.append(lane_id_cur)
    for per in t.findall('person'):
        edge_id_cur = per.attrib['edge']
        if edge_id_cur not in edge_lst:
            edge_lst.append(edge_id_cur)
            
#convert lane to edge from net file
lane_dict = {}
edge_net_lst = [] #get all edges from the network
for edge in net_root.findall('edge'):
    edge_id = edge.attrib['id']
    if edge_id not in edge_net_lst:
        edge_net_lst.append(edge_id)   
    for lane in edge.findall('lane'):
        lane_id = lane.attrib['id']
       # print(lane_id)
        if lane_id in lane_lst and lane_id not in lane_dict:
            lane_dict[lane_id] = edge_id
#print(lane_dict)

for edge in list(lane_dict.values()):
    if edge not in edge_lst:
        edge_lst.append(edge)
        

edges_missing_tazs = []
edges_with_tazs = list(edge_taz_dict.keys())
for edges in edge_lst:
    if edges not in edges_with_tazs and edges not in edges_missing_tazs:
        edges_missing_tazs.append(edges)
        
missings_taz_veh = []
missings_taz_ped_pub = []
for edges in edges_missing_tazs:
    if ":" in edges: 
        missings_taz_ped_pub.append(edges)
    else:
        missings_taz_veh.append(edges)
        
print('edge with tazs identified:', len(edges_with_tazs))
print('edge w.o tazs identified:', len(edges_missing_tazs))

#assign taz to missing edges
missing_edge_connection = {}
            
for c in net_root.findall('connection'):
    edge1 = c.attrib['from']
    edge2 = c.attrib['to']
    #print(edge1, edge2)
    
    if edge1 in edges_missing_tazs:
        #print(edge1)
        missing_edge_connection = assign_val(missing_edge_connection,
                                             edge1, edge2)
    elif edge2 in edges_missing_tazs:
        #print(edge2)
        missing_edge_connection = assign_val(missing_edge_connection,
                                            edge2, edge1)
        
#print(len(list(missing_edge_connection.keys())))
#print(len(edges_missing_tazs))
#previous return = 6329 (both)

print(len(list(missing_edge_connection.keys())))
print(len(edges_missing_tazs))
#previous return = 3203 (for 30 sec data)

#for edges in missing_edge_connection:
#print('test', convert2taz(missing_edge_connection['165086027#0']))

missing_edge_taz = {}
for edge in missing_edge_connection:
    missing_edge_taz[edge] = convert2taz(missing_edge_connection[edge])
    
#get edge with no taz mapped
edges_ = []
index_ = 0
for i in list(missing_edge_taz.values()):
    if i == -1:
        edges_.append(list(missing_edge_taz.keys())[index_])
    index_ += 1
    
for edge in missing_edge_taz:
    taz_val = missing_edge_taz[edge]
    if taz_val != -1:
        if edge not in edge_taz_dict:
            edge_taz_dict[edge] = taz_val
        else:
            print('edge in taz already', edge)   
            
#check '_' in lanes and edges
for i in edge_net_lst:
    if i[-2] == '_':
        pass
    if i[-3] == '_':
        pass
    if i[-4] == '_':
        print(i)


#get junction
missing_edge_junc = {}
#index_ = 0
for junc in net_root.findall('junction'):
    #print(index_)
    int_lane = junc.attrib['intLanes']
    inc_lane = junc.attrib['incLanes']
    junctionid = junc.attrib['id']

    int_inc_lane = []
    int_inc_lane.extend(sep_str(int_lane))
    int_inc_lane.extend(sep_str(inc_lane))
    #print(int_inc_lane)
    if len(int_inc_lane) != 0:
        edge_cur = get_edge(int_inc_lane, edge_net_lst)
    for edge in edge_cur:
        if edge in edges_:
            if edge not in missing_edge_junc:
                missing_edge_junc[edge] = assign_edge_val(edge_cur, 
                                                          junctionid, 
                                                          edge_taz_dict)            

print(len(missing_edge_junc))
selected_junc = list(missing_edge_junc.values())

index_ = 0
for edge in missing_edge_junc:
    tazs = missing_edge_junc[edge][:-1]
    #print(tazs)
    taz_ct = []
    unique_taz = np.unique(tazs)
    #print(unique_taz)
    for taz in unique_taz:
        taz_ct.append(tazs.count(taz))
    #print(taz_ct)
    try:
        taz_ = unique_taz[taz_ct.index(max(taz_ct))]
    except:
        taz_ = -1
    #print(taz_)
    if edge not in edge_taz_dict and taz_!=-1:
        edge_taz_dict[edge] = taz_
    elif taz_ == -1:
        print(edge)
#through the check, we assign 629 to the missing edges
#edge_taz_dict[':53217375_5'] = '629'

#missing_edge_junc[':53147426_0']
#edge_taz_dict[':cluster_gneJ449_gneJ451_1']
#edge_taz_dict[lane_dict['460423550#0_1']]

#get total taz in sumo network
taz_lst = []
for taz in taz_root.findall('taz'):
    taz_lst.append(taz.attrib['id'])
    
#def get_dt(fcd_outputpath, data_path, lane_dict, edge_taz_dict):
    #use the fcd output to check the data
#using 30 sec as an example
#fcd_outputpath = 'fcd_output_10per_all_30sec.xml'
#fcd_outputpath = 'fcd_output_10per_ht_30sec.xml'
#fcd_file


write_line = ''

#tree = ET.parse(fcd_outputpath)
#root = tree.getroot()
with open (fcd_file[:-3]+'csv', 'w') as f:
    for t in root.findall('timestep'):
        time_cur = t.attrib['time']
        #print(time_cur)
        #get lane id from vehicle
        for veh in t.findall('vehicle'):
            vehid = veh.attrib['id']
            #print(vehid)
            lane_id_cur = veh.attrib['lane']
            taz_id = edge_taz_dict[lane_dict[lane_id_cur]]
            write_line = vehid+','+taz_id+','+time_cur+'\n'
            f.write(write_line)
        try:    
            for per in t.findall('person'):
                perid = per.attrib['id']
                #print(perid)
                edge_id_cur = per.attrib['edge']
                taz_id = edge_taz_dict[edge_id_cur]
                write_line = perid+','+taz_id+','+time_cur+'\n'
                f.write(write_line)
        except: 
            pass
f.close()
print('data output done')     

In [7]:
#def get_dt(fcd_outputpath, data_path, lane_dict, edge_taz_dict):
    #use the fcd output to check the data

fcd_outputpath = 'fcd_output_5.xml'
write_line = ''
with open ('fcd_data_5.csv', 'w') as f:
    for t in root.findall('timestep'):
        time_cur = t.attrib['time']
        #print(time_cur)
        #get lane id from vehicle
        for veh in t.findall('vehicle'):
            vehid = veh.attrib['id']
            #print(vehid)
            lane_id_cur = veh.attrib['lane']
            taz_id = edge_taz_dict[lane_dict[lane_id_cur]]
            write_line = vehid+','+taz_id+','+time_cur+'\n'
            f.write(write_line)
        try:    
            for per in t.findall('person'):
                
                perid = per.attrib['id']
                #print(perid)
                edge_id_cur = per.attrib['edge']
                taz_id = edge_taz_dict[edge_id_cur]
                write_line = perid+','+taz_id+','+time_cur+'\n'
                f.write(write_line)
        except: 
            pass
f.close()
print('data output done')    

data output done


In [8]:
shpfilepath = 'G:/My Drive/2020/Bias/data-processing/spatial_check/'
zoneshp = gpd.read_file(shpfilepath+'Seattle_taz2010.shp')

In [9]:
fcd_out_30.sort_values(by=['time'])

NameError: name 'fcd_out_30' is not defined

In [ ]:
fcd_out_30.shape[0]-1

In [ ]:
def check_fcd_obs(fcd_outdt):
    test = fcd_outdt.groupby(['id']).size().to_frame()
    test.reset_index(inplace=True)
    
    test.columns =['id', 'num_trip']
    #print(test.sort_values(by=['num_trip'], ascending=False))
    print(test.num_trip.describe())
    test = test.sort_values(by=['num_trip'])
    return(test)
    
def revise_fcd_dt(fcd_outpath):
    fcd_outdt = pd.read_csv(fcd_outpath, header=None)
    fcd_outdt.columns = ['id', 'TAZ', 'time']
    fcd_outdt['time'] = fcd_outdt['time'].astype('int')
    fcd_outdt['hr'] = (fcd_outdt['time'].values/3600).astype(int)
    fcd_outdt['minus'] = (fcd_outdt['time'].values- fcd_outdt['hr'].values*3600)/60
    fcd_outdt['time_30'] = (fcd_outdt['time']/30).astype(int)
    fcd_outdt['time_5'] = (fcd_outdt['time']/300).astype(int)
    return(fcd_outdt)
    
def get_test_trace(fcd_outdt, id_info, zoneshp, plot_trajec=True):
    test_ = fcd_outdt[fcd_outdt['id'] == id_info]
    test_obs = test_.groupby(by=['TAZ']).sum()
    test_obs.reset_index(inplace=True)
    
    test_ = test_.sort_values(by=['id', 'time'])
    hr = test_['hr'].values
    minus = test_['minus'].values
    
    #check time span
    print('start time:', [hr[0], minus[0]])
    print('start time:', [hr[-1], minus[-1]])
    
    
    merged = zoneshp.set_index('TAZ').join(test_obs.set_index('TAZ'))
    merged.reset_index(inplace=True)
    merged['traj_index'] = 0
    merged['traj_index'][merged['TAZ'].isin(np.unique(test_.TAZ))] = 2
    merged['traj_index'][merged['time']==min(test_obs['time'])] = 1
    merged['traj_index'][merged['time']==max(test_obs['time'])] = 3
    
    if plot_trajec == True:
        cmap_style = plt.cm.get_cmap('Blues', 4)
        #selected_zone = np.unique(geoid_identify[:-4])
        fig, ax1 = plt.subplots(figsize=(16, 12))

        # set the range for the choropleth
        vmin, vmax = 0, 3
        variable = 'traj_index'
        merged.plot(column=variable, cmap=cmap_style, linewidth=0.8, \
                    ax=ax1, edgecolor='0.8')

        #cbar.set_yticklabels(['None', 'Starting zone', 'Passed', 'Ending zone'])
        ax3  = fig.add_axes([0.95,0.10,0.05,0.95])
        norm = mpl.colors.Normalize(vmin=vmin,vmax=vmax)
        cb1  = mpl.colorbar.ColorbarBase(ax3,cmap=cmap_style,norm=norm,orientation='vertical')
        cb1.set_ticks([vmax/8,vmax*3./8,vmax*5./8,vmax*7./8])
        cb1.set_ticklabels(['None', 'Starting zone', 'Passed', 'Ending zone'])
        #saving our map as .png file.

    

In [440]:
fcd_out = revise_fcd_dt('fcd_data_5.csv')


In [471]:
fcd_obs_ = check_fcd_obs(fcd_out_30)

count    70504.000000
mean        20.253049
std         16.835621
min          1.000000
25%          8.000000
50%         15.000000
75%         29.000000
max        277.000000
Name: num_trip, dtype: float64


In [477]:
3 in [3,4]

True

In [487]:
fcd_out_30['id']

0             10_41979657
1             13_49194874
2           21_47580428_0
3           24_47376848_0
4            2_49192917_0
                ...      
1427916             39828
1427917    480_5005_24409
1427918    503_5009_24418
1427919    480_5005_24409
1427920    480_5005_24409
Name: id, Length: 1427921, dtype: object

In [488]:
#try to find any connection between ped and drive
def check_id(split_id):
    if len(split_id[0]) in [3, 4] and len(split_id[1]) in [3, 4]:
        return(1)
    
id_veh = {}

for i in fcd_obs_['id'].values:
    if '_' in i:
        id_ = i.split('_')
        if len(id_)==3:
            if check_id(id_) == 1:
                id_veh[id_[2]] = i
                
id_ped = []
for i in fcd_obs_['id'].values:
    if '_' not in i:
        id_ped.append(i)

for i in id_ped:
    if i in id_veh:
        print(i, id_veh[i])
        
#after checking, it is found that the ped is doesn't match with the veh id

In [700]:
#check if we cover all the non-zero obs
np.count_nonzero(obs_mx) == len(loc_[0])

True

In [685]:
np.mean(obs_mx[np.where(obs_mx>0)])

4.134328358208955

In [499]:
time_5_obs['taz'] = time_5_obs['TAZ'].astype(int)

In [498]:
a = np.array([[1,2,3],[5,6,7]])
a.flatten()

array([1, 2, 3, 5, 6, 7])

In [ ]:
def get_stay(dt, time_period, time_p_col):
    dt[time_p_col] = (dt['time']/time_period).astype(int)
    dt = dt.sort_values(by=['id', 'time'])
    dt_id = dt['id'].values
    dt_time = dt['time_p_col'].values
    time_range = list(range(np.min(dt_time), np.max(dt_time)))
    

In [190]:
list(range(1, 5))

[1, 2, 3, 4]

In [181]:
3600*24

86400

In [185]:
86400/300

288.0

In [ ]:
time_loc = 

In [ ]:
#already in app_sim_method data
fcd_dt_path = 'fcd_data_5.csv'
#convert dt to matrix
def convert2mx(csv_path, savepath, save_index=False):
    fcd_out = pd.read_csv(fcd_dt_path, header=None)
    fcd_out.columns = ['id', 'taz', 'time']
    fcd_out.taz = fcd_out.taz.astype('int')
    fcd_out.time = fcd_out.time.astype('int')
    print('num id', len(np.unique(fcd_out.id)))

    #fcd_out.sort_values(by=['time', 'taz'])
    fcd_out['tl_id'] = fcd_out['taz'].astype(str)+'_'+((fcd_out['time']/300).astype(int)).astype(str)

    fcd_out = fcd_out[~fcd_out['time'].isin([86400, 86700])]
    time_ = np.unique((fcd_out['time']/300).astype(int))
    newuser_ = {}
    index_ = 0
    for i in np.unique(fcd_out.id):
        newuser_[i] = index_
        index_ += 1

    loc_time = {}
    index_ = 0
    taz_ = np.unique(fcd_out['taz'])
    time_ = np.unique((fcd_out['time']/300).astype(int))
    for taz in taz_:
        for t in time_:
            loc_time[str(taz)+'_'+str(t)] = index_
            index_ += 1

    origin_dt = np.zeros((len(newuser_), len(loc_time)))
    ids_ = fcd_out['id'].values
    tl_id_ = fcd_out['tl_id'].values
    for i in range(len(ids_)):
        origin_dt[newuser_[ids_[i]]][loc_time[tl_id_[i]]] = 1
    if save_index == True:
        pd.DataFrame(origin_dt).to_csv(savepath)
    else:
        return(origin_dt)